In [20]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
import java.util.Properties

val spark = SparkSession.builder().appName("Online Retail Analysis").getOrCreate()


spark = org.apache.spark.sql.SparkSession@35ca9c40


org.apache.spark.sql.SparkSession@35ca9c40

In [73]:
import spark.implicits._
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, DoubleType,DateType};



case class OnlineRetalDS (InvoiceNo: String, stockCode: Int, description: String, quantity: Int, 
                   invoiceDate: String, unitPrice: Double,customerID: String, country: String)

val OnlineRetailSchema = StructType( 
             List(
            StructField("InvoiceNo", StringType, true),
            StructField("stockCode", IntegerType, true),
            StructField("description", StringType, true),
            StructField("quantity", IntegerType, true),
            StructField("invoiceDate", StringType, true),
            StructField("unitPrice", DoubleType, true),
            StructField("customerID", StringType, true),
            StructField("country", StringType, true)
            ))

                 

defined class OnlineRetalDS
OnlineRetailSchema = StructType(StructField(InvoiceNo,StringType,true), StructField(stockCode,IntegerType,true), StructField(description,StringType,true), StructField(quantity,IntegerType,true), StructField(invoiceDate,StringType,true), StructField(unitPrice,DoubleType,true), StructField(customerID,StringType,true), StructField(country,StringType,true))


StructType(StructField(InvoiceNo,StringType,true), StructField(stockCode,IntegerType,true), StructField(description,StringType,true), StructField(quantity,IntegerType,true), StructField(invoiceDate,StringType,true), StructField(unitPrice,DoubleType,true), StructField(customerID,StringType,true), StructField(country,StringType,true))

In [74]:
val OnlineRetailDF = spark.read.format("csv")
.option("header","true")
.option("timestampFormat","dd/MM/yy HH:mm")
.schema(OnlineRetailSchema)
.load("/home/jignesh/Downloads/OnlineRetailNew.csv")




OnlineRetailDF.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|stockCode|         description|quantity|   invoiceDate|unitPrice|customerID|       country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|     null|     null|                null|    null|          null|     null|      null|          null|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|     null|     null|                null|    null|          null|     null|      null|          null|
|     null|     null|                null|    null|          null|     null|      null|          null|
|     null|     null|                null|    null|          null|     null|      null|          null|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

OnlineRetailDF = [InvoiceNo: string, stockCode: int ... 6 more fields]


[InvoiceNo: string, stockCode: int ... 6 more fields]

In [75]:
val OnlineRetailDF1 = OnlineRetailDF.filter('InvoiceNo =!= "null")

OnlineRetailDF1 = [InvoiceNo: string, stockCode: int ... 6 more fields]


[InvoiceNo: string, stockCode: int ... 6 more fields]

In [76]:
import org.apache.spark.sql.functions._

val OnlineRetailTS = OnlineRetailDF1.withColumn("ts",
                                                unix_timestamp($"InvoiceDate","dd/MM/yy HH:mm").cast("timestamp"))

OnlineRetailTS.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
|InvoiceNo|stockCode|         description|quantity|   invoiceDate|unitPrice|customerID|       country|                 ts|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|2010-01-12 08:26:00|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.25|     17850|United Kingdom|2010-01-12 08:26:00|
|   536366|    22633|HAND WARMER UNION...|       6|12/1/2010 8:28|     1.85|     17850|United Kingdom|2010-01-12 08:28:00|
|   536366|    22632|HAND WARMER RED P...|       6|12/1/2010 8:28|     1.85|     17850|United Kingdom|2010-01-12 08:28:00|
|   536367|    8

OnlineRetailTS = [InvoiceNo: string, stockCode: int ... 7 more fields]


[InvoiceNo: string, stockCode: int ... 7 more fields]

In [77]:

OnlineRetailTS.createOrReplaceTempView("RetailTable")



In [87]:
val r3DF= spark.sql("select * from RetailTable where ts <'2011-12-01'")
val r4DF= spark.sql("select * from RetailTable where ts >'2011-12-01'")

//spark.sql("select customerID, count(*) from RetailTable group by customerID order by count(*) desc").show()

r3DF = [InvoiceNo: string, stockCode: int ... 7 more fields]
r4DF = [InvoiceNo: string, stockCode: int ... 7 more fields]


[InvoiceNo: string, stockCode: int ... 7 more fields]

In [89]:
val selectdata = r4DF.select("InvoiceNo","stockCode","description","quantity","unitPrice"
                             ,"customerID","country","ts")
val writedata = selectdata.withColumnRenamed("ts","invoiceDate")
writedata.show()

+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+
|InvoiceNo|stockCode|         description|quantity|unitPrice|customerID|       country|        invoiceDate|
+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+
|   540848|    90204|PAIR OF ENAMEL BU...|       1|     3.39|      null|United Kingdom|2011-12-01 09:26:00|
|   540848|    21043|APRON MODERN VINT...|       1|     4.21|      null|United Kingdom|2011-12-01 09:26:00|
|   540848|    21520|BOOZE & WOMEN GRE...|       2|     0.43|      null|United Kingdom|2011-12-01 09:26:00|
|   540848|    21707|FOLDING UMBRELLA ...|       1|     3.36|      null|United Kingdom|2011-12-01 09:26:00|
|   540848|    21884|CAKES AND BOWS GI...|       7|     0.43|      null|United Kingdom|2011-12-01 09:26:00|
|   540848|    22144|CHRISTMAS CRAFT L...|       1|     4.21|      null|United Kingdom|2011-12-01 09:26:00|
|   540848|    22219|LOVEBIR

selectdata = [InvoiceNo: string, stockCode: int ... 6 more fields]
writedata = [InvoiceNo: string, stockCode: int ... 6 more fields]


[InvoiceNo: string, stockCode: int ... 6 more fields]

In [91]:
val dbURL = "jdbc:mysql://localhost:3306/TestDB?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=UTC"
val prop = new Properties()
prop.setProperty("user","jignesh")
prop.setProperty("password","Data1234")
writedata.write.mode("append").jdbc(dbURL,"transactions",prop)

dbURL = jdbc:mysql://localhost:3306/TestDB?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=UTC
prop = {user=jignesh, password=Data1234}


lastException: Throwable = null


{user=jignesh, password=Data1234}

In [92]:
writedata.select("*").write.format("json").save("hdfs://localhost:9000/users/OnllineRetail")

Name: org.apache.spark.SparkException
Message: Job aborted.
StackTrace:   at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.